In order to successfully complete this assignment you need to participate both individually and in groups during class.   Have one of the instructors check your notebook and sign you out before leaving class. Turn in your assignment using D2L. 

---


# ICA 20: CUDA Blocks/Grids/Threads

<img alt="image of a greek phalax of fighters. Designed to represent the marching forward of a CUDA WARP" src="https://upload.wikimedia.org/wikipedia/commons/e/ed/Greek_Phalanx.jpg">


Image from: https://en.wikipedia.org/wiki/Phalanx
    


### Agenda for today's class (70 minutes)

1. (20 minutes) [Pre class Review](#Pre-class-Review)
2. (50 minutes) [1D Wave Example](#Start-1D-Wave-Example)



---
<a name=Pre-class-Review></a>
# 1. Pre-class Review

&#9989; **<font color=red>DO THIS:</font>** Discuss with your group the process for indexing a 2D array as was discussed in the PCA. 



<img alt="Visual representation of how a grid is made up of blocks which is then made of of a warp" src="https://www.researchgate.net/profile/Nandakishore_Santhi/publication/285235201/figure/fig2/AS:352267598352385@1460998548443/Example-of-a-CUDA-grid-The-grid-is-composed-of-blocks-each-block-is-composed-of-warps.png" width=50%>



---
<a name=Start-1D-Wave-Example></a>
# 2. 1D Wave Example

See if you can adapt the 1D wave code to work using CUDA. Benchmark both the CPU and GPU versions and see if you can get a speedup. 


```c++
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int main(int argc, char ** argv) {
    int nx = 500;
    int nt = 100000;
    int i,it;
    double x[nx];
    double y[nx];
    double v[nx];
    double dvdt[nx];
    double dt;
    double dx;
    double max,min;
    double dx2inv;
    double tmax;
    int nxm1;

    max=10.0;
    min=0.0;
    dx = (max-min)/(double)(nx);
    x[0] = min;
    for(i=1;i<nx-1;i++) {
        x[i] = min+(double)i*dx;
    }
    x[nx-1] = max;
    tmax=10.0;
    dt= (tmax-0.0)/(double)(nt);

    for (i=0;i<nx;i++)  {
        y[i] = exp(-(x[i]-5.0)*(x[i]-5.0));
        v[i] = 0.0;
        dvdt[i] = 0.0;
    }
    
    dx2inv=1.0/(dx*dx);
    nxm1=nx-1;
    for(it=0;it<nt-1;it++) {
        for(i=1;i<nxm1;i++)
            dvdt[i]=(y[i+1]+y[i-1]-2.0*y[i])*(dx2inv);

        for(i=1; i<nxm1; i++)  {
            v[i] = v[i] + dt*dvdt[i];
            y[i] = y[i] + dt*v[i];
        }

    }

    for(i=nx/2-10; i<nx/2+10; i++) {
        printf("%g %g\n",x[i],y[i]);
    }

    return 0;
}

```

```c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

#define NX 500
#define NT 100000

__global__ void compute_dvdt(double *y, double *dvdt, double dx2inv, int nxm1) {
    int i = blockIdx.x * blockDim.x + threadIdx.x + 1;
    if (i < nxm1) {
        dvdt[i] = (y[i+1] + y[i-1] - 2.0 * y[i]) * dx2inv;
    }
}

__global__ void update_values(double *y, double *v, double *dvdt, double dt, int nxm1) {
    int i = blockIdx.x * blockDim.x + threadIdx.x + 1;
    if (i < nxm1) {
        v[i] += dt * dvdt[i];
        y[i] += dt * v[i];
    }
}

void initialize_arrays(double *x, double *y, double *v, double *dvdt, double min, double dx) {
    for (int i = 0; i < NX; i++) {
        x[i] = min + i * dx;
        y[i] = exp(-(x[i] - 5.0) * (x[i] - 5.0));
        v[i] = 0.0;
        dvdt[i] = 0.0;
    }
}

int main() {
    double *x, *y, *v, *dvdt;
    double *d_y, *d_v, *d_dvdt;
    double dt, dx, dx2inv;
    double max = 10.0, min = 0.0;
    int nxm1 = NX - 1;
    
    // Allocate host memory
    x = (double*)malloc(NX * sizeof(double));
    y = (double*)malloc(NX * sizeof(double));
    v = (double*)malloc(NX * sizeof(double));
    dvdt = (double*)malloc(NX * sizeof(double));

    // Grid spacing
    dx = (max - min) / (double)(NX);
    dx2inv = 1.0 / (dx * dx);
    dt = 10.0 / (double)(NT);

    // Initialize arrays
    initialize_arrays(x, y, v, dvdt, min, dx);

    // Allocate device memory
    cudaMalloc((void**)&d_y, NX * sizeof(double));
    cudaMalloc((void**)&d_v, NX * sizeof(double));
    cudaMalloc((void**)&d_dvdt, NX * sizeof(double));

    // Copy data to device
    cudaMemcpy(d_y, y, NX * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_v, v, NX * sizeof(double), cudaMemcpyHostToDevice);

    // Define CUDA kernel launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (NX + threadsPerBlock - 1) / threadsPerBlock;

    // Time stepping loop
    for (int it = 0; it < NT - 1; it++) {
        compute_dvdt<<<blocksPerGrid, threadsPerBlock>>>(d_y, d_dvdt, dx2inv, nxm1);
        update_values<<<blocksPerGrid, threadsPerBlock>>>(d_y, d_v, d_dvdt, dt, nxm1);
        cudaDeviceSynchronize();
    }

    // Copy results back to host
    cudaMemcpy(y, d_y, NX * sizeof(double), cudaMemcpyDeviceToHost);

    // Free memory
    free(x); free(y); free(v); free(dvdt);
    cudaFree(d_y); cudaFree(d_v); cudaFree(d_dvdt);

    return 0;
}
```
Serial: .345s 
CUDA: .052s 


-----
### Congratulations, we're done!

Have one of the instructors check your notebook and sign you out before leaving class. Turn in your assignment using D2L.

Written by Dr. Dirk Colbry, Michigan State University (Updated by Dr. Nathan Haut in Spring 2025)
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----